In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url = 'https://caelum-online-public.s3.amazonaws.com/2929-pandas/dataset-telecon.json'

In [ ]:
dados = pd.read_json(url)

In [ ]:
dados.head()

In [ ]:
dados.info()

In [ ]:
dados_churn = dados

In [ ]:
dados.head()

In [ ]:
dados_churn

In [ ]:
dados_churn['conta'][0]

In [ ]:
pd.json_normalize(dados_churn['conta']).head()

In [ ]:
pd.json_normalize(dados_churn['telefone']).head()

In [ ]:
import json

In [ ]:
with open('dataset-telecon.json') as f:
  json_bruto = json.load(f)

In [ ]:
json_bruto

In [ ]:
dados_normalizados = pd.json_normalize(json_bruto)
dados_normalizados.head()


In [ ]:
dados_normalizados.info()

In [ ]:
# dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)
# esse comando deu erro pois os valores da coluna conta.cobranca.total possuem caracteres que nao da para converter normalmente para string

In [ ]:
dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '].head()

In [ ]:
dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '][
    ['cliente.tempo_servico', 'conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']
]

In [ ]:
idx = dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '].index

In [ ]:
dados_normalizados.loc[idx, "conta.cobranca.Total"] = dados_normalizados.loc[idx, "conta.cobranca.mensal"] * 24

In [ ]:
dados_normalizados.loc[idx, "cliente.tempo_servico"] = 24

In [ ]:
dados_normalizados.loc[idx][['cliente.tempo_servico', 'conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']
]

In [ ]:
dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)

In [ ]:
dados_normalizados.info()

In [ ]:
for col in dados_normalizados.columns:
  print(f'coluna: {col}')
  print(dados_normalizados[col].unique())
  print('-' * 30)

In [ ]:
dados_normalizados.query('Churn == ""')

In [ ]:
dados_sem_vazio = dados_normalizados[dados_normalizados['Churn'] != ''].copy()

In [ ]:
dados_sem_vazio.info()

Só aplcando o metodo reset index, ele vai criar uma nova coluna de index, e preservar os indices antigos em uma coluna extra. NO PROXIMO CAMPO DE CODIGO VAMOS REMOVER ESSA COLUNA EXTRA COM OS INDICES ANTIGOS

In [ ]:
dados_sem_vazio.reset_index()

Inplace vai alterar o dados_sem_vazio DIRETAMENTE sem precisar atribuir assim -> (     dados_sem_vazio = dados_sem_vazio.reset_index(drop = True, inplace = True)    )

In [ ]:
dados_sem_vazio.reset_index(drop = True, inplace = True)

retorna uma series booleana , se na linha do dataframe há quaiser dados duplicados ou seja dados que sejam exatamente iguais a qualquer outra linha no dataframe(todas as colunas com o mesmo valor)

In [ ]:
dados_sem_vazio.duplicated()


In [ ]:
dados_sem_vazio.duplicated().sum()


In [ ]:
filtro_duplicadas = dados_sem_vazio.duplicated().sum()


In [ ]:
filtro_duplicadas = dados_sem_vazio.duplicated()
filtro_duplicadas

In [ ]:
dados_sem_vazio[filtro_duplicadas]

novamente usando o inplace para atribuir automaticamente

In [ ]:
dados_sem_vazio.drop_duplicates(inplace=True)

In [ ]:
dados_sem_vazio.duplicated()

In [ ]:
dados_sem_vazio.duplicated().sum()

IDENTIFICANDO E EXCLUINDO DADOS NULOS

In [ ]:
dados_sem_vazio.isna()

In [ ]:
dados_sem_vazio.isna().sum()

In [ ]:
dados_sem_vazio.isna().sum().sum()

In [ ]:
dados_sem_vazio[dados_sem_vazio.isna().any(axis=1)]

In [ ]:
filtro = dados_sem_vazio['cliente.tempo_servico'].isna()

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
import numpy as np
np.ceil(5957.90 / 90.45)

In [ ]:
dados_sem_vazio['cliente.tempo_servico'].fillna(
    np.ceil(
        dados_sem_vazio['conta.cobranca.Total'] / dados_sem_vazio['conta.cobranca.mensal']
    ), inplace=True
)

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
dados_sem_vazio.isna().sum()

In [ ]:
dados_sem_vazio['conta.contrato'].value_counts()

In [ ]:
colunas_dropar = ['conta.contrato', 'conta.faturamente_eletronico', 'conta.metodo_pagamento']

In [ ]:
dados_sem_vazio[colunas_dropar].isna()

In [ ]:
dados_sem_vazio[colunas_dropar].isna().any(axis=1).sum()

subset é para passar as colunas para avaliacao dos dados nulos

In [ ]:
df_sem_nulo = dados_sem_vazio.dropna(subset=colunas_dropar).copy()
df_sem_nulo.head()

In [ ]:
df_sem_nulo.reset_index(drop = True, inplace = True)

In [ ]:
df_sem_nulo.isna().sum()

In [ ]:
df_sem_nulo.describe()

In [ ]:
import seaborn as sns


In [ ]:
sns.boxplot(x=df_sem_nulo['cliente.tempo_servico'])

In [ ]:
Q1 = df_sem_nulo['cliente.tempo_servico'].quantile(.25)
Q3 = df_sem_nulo['cliente.tempo_servico'].quantile(.75)
IQR = Q3 - Q1
limite_inferior = Q1 -1.5*IQR
limite_superior = Q1 +1.5*IQR

In [ ]:
type(Q1)

In [ ]:
outliers_index = (df_sem_nulo['cliente.tempo_servico'] < limite_inferior) | (df_sem_nulo['cliente.tempo_servico'] > limite_superior)
outliers_index

In [ ]:
df_sem_nulo[outliers_index]['cliente.tempo_servico']

In [ ]:
df_sem_out = df_sem_nulo.copy()

In [ ]:
df_sem_out[outliers_index]['cliente.tempo_servico']

In [ ]:
df_sem_out.loc[outliers_index , 'cliente.tempo_servico'] = np.ceil(
    df_sem_out.loc[outliers_index , 'conta.cobranca.Total'] /
     df_sem_out.loc[outliers_index , 'conta.cobranca.mensal']

)

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
df_sem_out[outliers_index][['cliente.tempo_servico' , 'conta.cobranca.mensal' , 'conta.cobranca.Total' ]]


In [ ]:
df_sem_out[outliers_index]['cliente.tempo_servico']

In [ ]:
Q1 = df_sem_out['cliente.tempo_servico'].quantile(.25)
Q3 = df_sem_out['cliente.tempo_servico'].quantile(.75)
IQR = Q3 - Q1
limite_inferior = Q1 -1.5*IQR
limite_superior = Q1 +1.5*IQR

outliers_index = (df_sem_out['cliente.tempo_servico'] < limite_inferior) | (df_sem_out['cliente.tempo_servico'] > limite_superior)
outliers_index

In [ ]:
df_sem_out[outliers_index]

In [ ]:
df_sem_out = df_sem_out[~outliers_index]
df_sem_out

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
df_sem_out.reset_index(drop=True, inplace = True)

In [ ]:
df_sem_out

In [ ]:
df_sem_out.drop('id_cliente', axis=1)

In [ ]:
df_sem_id = df_sem_out.drop('id_cliente', axis=1).copy()
df_sem_id

In [ ]:
mapeamento = {
    'nao': 0,
    'sim': 1,
    'masculino': 0,
    'feminino': 1
}

In [ ]:
for col in df_sem_id.columns:
    print(f"Coluna: {col}")
    print(df_sem_id[col].unique())
    print("-" * 30)

In [ ]:
colunas = ['telefone.servico_telefone', 'Churn', 'cliente.parceiro', 'cliente.dependentes', 'conta.faturamente_eletronico', 'cliente.genero']

In [ ]:
df_sem_id[colunas] = df_sem_id[colunas].replace(mapeamento)
df_sem_id


In [ ]:
for col in df_sem_id.columns:
    print(f"Coluna: {col}")
    print(df_sem_id[col].unique())
    print("-" * 30)

In [ ]:
pd.get_dummies(df_sem_id)

In [ ]:
df_dummies = pd.get_dummies(df_sem_id, dtype=int).copy()
df_dummies.head()

In [ ]:
df_dummies.info()